In [1]:
import pandas as pd
import numpy as np



In [2]:

df = (pd.read_csv("../causal-inference-in-python/data/online_mkt.csv")
      .astype({"date":"datetime64[ns]"})
      .query("post==0"))

In [3]:
df.head()

,app_download,population,city,state,date,post,treated
0,3066.0,12396372,sao_paulo,sao_paulo,2022-03-01,0,1
1,2701.0,12396372,sao_paulo,sao_paulo,2022-03-02,0,1
2,1927.0,12396372,sao_paulo,sao_paulo,2022-03-03,0,1
3,1451.0,12396372,sao_paulo,sao_paulo,2022-03-04,0,1
4,1248.0,12396372,sao_paulo,sao_paulo,2022-03-05,0,1


# Computing variance and power calculations

In [4]:
import polars as pl

In [5]:
dfpl = pl.DataFrame(df)

In [22]:
dfpl.head()

app_download,population,city,state,date,post,treated
f64,i64,str,str,datetime[ns],i64,i64
3066.0,12396372,"""sao_paulo""","""sao_paulo""",2022-03-01 00:00:00,0,1
2701.0,12396372,"""sao_paulo""","""sao_paulo""",2022-03-02 00:00:00,0,1
1927.0,12396372,"""sao_paulo""","""sao_paulo""",2022-03-03 00:00:00,0,1
1451.0,12396372,"""sao_paulo""","""sao_paulo""",2022-03-04 00:00:00,0,1
1248.0,12396372,"""sao_paulo""","""sao_paulo""",2022-03-05 00:00:00,0,1


In [20]:
dfpl.mean()

app_download,population,city,state,date,post,treated
f64,f64,str,str,datetime[ns],f64,f64
160.04,1.3701e6,null,null,2022-03-31 00:00:00,0.0,0.06


In [50]:
varr =np.var(
    dfpl
    # .filter(pl.col("treated")==0)
    .group_by("city")
    .agg((pl.mean('app_download')).alias("mean_app_dnld"))
    .select("mean_app_dnld")
    .to_numpy()
)
varr

np.float64(143789.36064670788)

In [51]:
## Overall variance
# MDE of 5%
baseline = (
    np.mean(
        dfpl
        .group_by("city")
        .agg((pl.mean("app_download").alias("mean_dnld")))
        .select("mean_dnld")
        .to_numpy()
    )
)

baseline

np.float64(160.04000000000002)

In [52]:
exp_uplift = baseline * 1.05
exp_uplift

np.float64(168.04200000000003)

In [53]:
##  Number of cities needed
N = (16.0*varr)/((baseline * 0.05)**2)
N

np.float64(35929.3732294764)